# Useful references

## Python + Numpy + Matplotlib + etc.

* Python Numpy Tutorial: http://cs231n.github.io/python-numpy-tutorial/
* Computational Statistics in Python: http://people.duke.edu/~ccc14/sta-663-2017/
* Numpy for MATLAB users: https://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html
* MATLAB synonymous commands in Python/NumPy: http://mathesaurus.sourceforge.net/
* Effectively using matplotlib: https://pbpython.com/effective-matplotlib.html


## NEURON (with Python)
* NEURON documentation: https://www.neuron.yale.edu/neuron/static/py_doc/index.html
* NEURON + Python tutorial: https://neuron.yale.edu/neuron/docs/scripting-neuron-basics


# 1. Current clamp simulation on a passive pyramidal neuron

Here we demonstrate how to run a current clamp simulation with a passive pyramidal neuron for the second home work. First, we begin by importing a module `libcell` and constructs a pyramidal cell in NEURON

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from neuron import h, gui
import libcell

In [ ]:
cell = libcell.L23()

In [ ]:
h.psection()

Here we load a session file for the virtual current clamp experiment:

In [ ]:
h.xopen("iclamp.ses")

Loading this session embeds a current clamp electrode `h.IClamp[0]`, whose parameters are:

In [ ]:
print('Activation onset = {} ms\nDuration = {} ms\nAmplitude = {} nA'.format(
        h.IClamp[0].delay, h.IClamp[0].dur, h.IClamp[0].amp))

You can control the location and parameters via the "PointProcessManager" window. Now let's run simulation while recording the membrane potential at soma.

In [ ]:
def run(tstop=300):
    """runs a simulation and returns the membrane potential recording at the soma."""
    dt = 0.1 # Again we use 10kHz sampling rate, e.g. 0.1 ms interval
    vrec = h.Vector() # The recording will be save in this vector
    vrec.record(h.soma(0.5)._ref_v, dt)
    
    h.tstop = tstop  # Set how long the simulation will run.
    h.init()
    h.run()
    
    return vrec.c() # Should return a copy of the vector


Let's collect multiple recordings in a list and also make a note of where the electrode was:

In [ ]:
h.distance(sec=cell.soma) # Set an origin to cell.soma(0)
h.distance(h.IClamp[0].get_segment()) # Measure a distance to the origin from the electrode

In [ ]:
# Reset lists
vrecs = []
distances = []

In [ ]:
result = run()
vrecs.append(result)
# r18_1.distance_from_soma(ic) computes a distance from a soma 
# to the current clamp electrode
distances.append(r18_1.distance_from_soma(h.IClamp[0]))

print("Number of simulations ran =", len(distances))

In [ ]:
%matplotlib inline

fig,ax = plt.subplots()
t = np.arange(2000)*0.1
for v in vrecs:
    ax.plot(t, v)
ax.legend(distances)
ax.set(xlabel='time (ms)', ylabel='Voltage (mV)')

## * How to locate and move point processes

* Help for point processes: https://www.neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/mechanisms/mech.html

To find out where our electrode is now:

In [ ]:
h.IClamp[0].get_segment()

In [ ]:
h.distance(sec=cell.soma) # Set an origin to cell.soma(0)
h.distance(h.IClamp[0].get_segment()) # Measure a distance to the origin from the electrode

Let's go through all sections and segments:

In [ ]:
for sec in h.allsec():
    for seg in sec.allseg():
        print(sec.name(), seg.x)

With distances:

In [ ]:
for sec in h.allsec():
    for seg in sec.allseg():
        segname = '{}({})'.format(sec.name(), seg.x)
        h.distance(sec=cell.soma)
        print(segname, '\t', h.distance(seg))

`PointProcess.loc(segment)` moves the point process to a given segment.

**Exercise:** Find a segment in `cell` by a name and move our electrode to it.